In [1]:
import pandas as pd

In [2]:
usvideos=pd.read_csv("USvideos.csv")

In [26]:
mp=usvideos[["video_id","category_id"]].drop_duplicates()

In [31]:
for k,v in zip(list(mp["video_id"]),list(mp["category_id"])):
    try:
        os.system("mv /data/youtubetn/%s.jpg /data/youtubetn/%s/%s.jpg"%(k,v,k))
    except:
        pass

In [4]:
tn_list = usvideos.thumbnail_link.unique().tolist()
len(tn_list)

4547

In [5]:
cat_list = usvideos.category_id.unique()
len(cat_list)

16

In [6]:
%mkdir -p /data/youtubetn
%cd /data/youtubetn

/data/youtubetn


In [7]:
import os
def dltn(url):
    fname=url.split("/")[-2]+"."+url.split(".")[-1]
    os.system("wget -O /data/youtubetn/%s %s"%(fname,url))

In [8]:
from multiprocessing import Pool

In [ ]:
p=Pool(6)
p.map(dltn,tn_list)

In [52]:
from keras.layers import *
from keras.models import *

In [10]:
from p3self.bcolzer import bcolzer

In [11]:
bc=bcolzerer("img")

[task:bcolzer_task>>start]<2018-03-10_16:04:39|0s,0s>	
[starting]<2018-03-10_16:04:39|0s,0s>	for data type [img]
[type_set]<2018-03-10_16:04:39|0s,0s>	type set to "img", use self.img_gen(image_dir) to create the generator
[set_roll_process_func]<2018-03-10_16:04:39|0s,0s>	
                    default roll_func
                    return the 1st element of the set
                    x:input data
                    


In [65]:
bc.img_gen("/data/youtubetn/",with_class=True,img_shape=(120,90))

Found 4449 images belonging to 16 classes.


In [66]:
from PIL import Image

In [67]:
preproc = lambda x: (x-np.array([123.68,116.779,103.939],dtype=np.float32))[:,:,:,::-1]*0.017

In [68]:
ipt=Input((120,90,3))
x=Lambda(preproc)(ipt)
x=Conv2D(64,3,padding="same",activation="relu")(x)
x=Conv2D(64,3,padding="same",activation="relu")(x)
x=Conv2D(64,3,strides=(2,2),padding="same",activation="relu")(x)
x=BatchNormalization()(x)
x=Conv2D(64,3,padding="same",activation="relu")(x)
x=Conv2D(64,3,padding="same",activation="relu")(x)
x=Conv2D(64,3,strides=(2,2),padding="same",activation="relu")(x)
x=BatchNormalization()(x)
x=Conv2D(128,3,padding="same",activation="relu")(x)
x=Conv2D(128,3,padding="same",activation="relu")(x)
x=Conv2D(128,3,strides=(2,2),padding="same",activation="relu")(x)
x=BatchNormalization()(x)
x=Conv2D(128,3,padding="same",activation="relu")(x)
x=Conv2D(128,3,padding="same",activation="relu")(x)
x=Conv2D(128,3,strides=(2,2),padding="same",activation="relu")(x)
x=BatchNormalization()(x)
x=Conv2D(128,3,padding="same",activation="relu")(x)
x=Conv2D(128,3,padding="same",activation="relu")(x)
x=Conv2D(128,3,strides=(2,2),padding="same",activation="relu")(x)
x=BatchNormalization()(x)
x=Flatten()(x)
x=Dense(1024,activation="relu")(x)
x=Dense(16,activation="softmax")(x)
md=Model(ipt,x)

In [69]:
md.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_11 (InputLayer)        (None, 120, 90, 3)        0         
_________________________________________________________________
lambda_9 (Lambda)            (None, 120, 90, 3)        0         
_________________________________________________________________
conv2d_82 (Conv2D)           (None, 120, 90, 64)       1792      
_________________________________________________________________
conv2d_83 (Conv2D)           (None, 120, 90, 64)       36928     
_________________________________________________________________
conv2d_84 (Conv2D)           (None, 60, 45, 64)        36928     
_________________________________________________________________
batch_normalization_28 (Batc (None, 60, 45, 64)        256       
_________________________________________________________________
conv2d_85 (Conv2D)           (None, 60, 45, 64)        36928     
__________

In [70]:
md.compile(loss="categorical_crossentropy",optimizer="Adam",metrics=["accuracy"])

In [71]:
md.fit_generator(bc.gen,steps_per_epoch=int(bc.gen.n/bc.gen.batch_size),epochs=1)

Epoch 1/1
139/139 [==============================] - 1236s 9s/step - loss: 2.5001 - acc: 0.2239


In [74]:
md.predict(bc.gen.next()[0]).argmax(axis=-1)

array([ 9,  9,  9,  9,  9,  1,  9,  1,  9,  9,  9,  9,  9,  1,  1,  9,  9,
        9,  9,  9,  9,  9, 11,  9,  9,  9,  9,  9,  9,  9,  1,  9])

In [75]:
bc.gen.class_indices

{'1': 0,
 '10': 1,
 '15': 2,
 '17': 3,
 '19': 4,
 '2': 5,
 '20': 6,
 '22': 7,
 '23': 8,
 '24': 9,
 '25': 10,
 '26': 11,
 '27': 12,
 '28': 13,
 '29': 14,
 '43': 15}